## DETERMINING HABITABLE ZONES ##

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from astropy import constants as const
from astropy import units as u

# DEFYNING CONSTANTS

In [21]:
# Definiendo constantes
G = const.G.to(u.cm**3/(u.g*u.s**2)) # Constante Gravitacional en cm^3/(g*s^2)
Msol = const.M_sun.to(u.g) # Masa solar en gramos
R_sol = const.R_sun.to(u.cm)  # Radio del sol en centímetros

In [22]:
# Read the CSV file with space as the delimiter
exoplanet_data = pd.read_csv('exoplanet_data.csv', sep=',', comment='#')
exoplanet_data

,Planet_name,Host_name,gaia_id,Number_of_stars,Number_of_planets,Number_of_moons,disc_facility,Planet_controversial_flag,Orbital_period[days],delta_orbpererr1,...,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,st_nspec
0,K2-111 b,K2-111,Gaia DR2 53006669599267328,1,2,0,K2,0,5.351800,4.000000e-04,...,11.130,0.034,-0.034,9.377000,0.021,-0.021,10.92940,0.000621,-0.000621,0
1,TOI-1695 b,TOI-1695,Gaia DR2 534988616816537728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,3.134279,7.100000e-06,...,12.989,0.081,-0.081,8.818000,0.021,-0.021,12.13640,0.000460,-0.000460,0
2,K2-170 b,K2-170,Gaia DR2 2599451994385832448,1,2,0,K2,0,7.576544,1.841000e-03,...,12.668,0.114,-0.114,11.075000,0.021,-0.021,12.49590,0.000222,-0.000222,0
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,15.030,0.229,-0.229,12.782000,0.022,-0.022,14.85610,0.000306,-0.000306,0
4,Kepler-1398 b,Kepler-1398,Gaia DR2 2128776801985730048,1,2,0,Kepler,0,2.788157,2.021000e-05,...,13.752,0.103,-0.103,12.355000,0.021,-0.021,13.66050,0.000228,-0.000228,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Kepler-99 b,Kepler-99,Gaia DR2 2076871091425583232,2,1,0,Kepler,0,4.603580,NaN,...,13.206,0.092,-0.092,10.757000,0.018,-0.018,12.96180,0.000373,-0.000373,0
701,HIP 29442 d,HD 42813,Gaia DR2 2993561629444856960,1,3,0,Paranal Observatory,0,6.429575,2.600000e-05,...,9.490,0.030,-0.030,7.587000,0.024,-0.024,9.27468,0.000294,-0.000294,0
702,TOI-1467 b,TOI-1467,Gaia DR2 403141126645929728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,5.971147,8.746000e-06,...,12.293,0.017,-0.017,8.568000,0.019,-0.019,11.56220,0.000875,-0.000875,0
703,EPIC 201595106 b,EPIC 201595106,Gaia DR2 3701123406596146048,1,2,0,K2,0,0.877240,4.000000e-05,...,11.746,0.069,-0.069,10.218000,0.021,-0.021,11.64770,0.000245,-0.000245,0


## Calculando zonas de habitabilidad 

Basándonos en el modelo de  (Kopparapu,2013), el cual considera planetas rocosos que tienen una masa menor a $5M_{\oplus}$,donde se asumen atmósferas dominadas por $H_2O$ para el límite interno y $CO{-1}_2$ para el límite externo. 
Este modelo es un modelo unidimensional, en donde se asume la atmósfera como capas, apiladas unas sobre otras. En el modelo planteado por los autores, se ignoran factores como las nubes y se concentran en como el calor se mueve a través de la atmósfera por radiación(proveniente del sol) y la convección, producto del aumento del aire caliente. 
Para este modelo se tuvieron en cuenta los siguientes casos: 1. Toman un planeta fijo y estudian el efecto de los gases no condensables($N_2$) en los límites de la ZH, variando la presión parcial de $N_2$; 2 Fijar una presión debida al $N_2$ de 0.01 bar para distintas masas de planetas entre el intervalo ya mencionado, con el fin de estudiar el efecto de la gravedad por sí solo, y 3 la presión del $N_2$ se ajusta de acuerdo al tamaño del radio del planeta y verificar el efecto del tamaño del planeta en la abundancia volátil.



# MÉTODO 
En el artículo mencionado, se tomaron temperaturas efectivas entre $2500 k\leq T_eff \leq 7200 k$. Para esta primera parte se tomarán los planetas dentro del rango de masa y se tomarán únicamente aquellos planetas cuya estrella sea de secuencia principal.

In [23]:
# Obtaining plaets with Mass between 0.4 to 5 and then main sequence stars 
kopparu = exoplanet_data[
    (exoplanet_data['Planet_mass[Earth]'] >= 0.4) & 
    (exoplanet_data['Planet_mass[Earth]'] <= 5)
]

kopparu

,Planet_name,Host_name,gaia_id,Number_of_stars,Number_of_planets,Number_of_moons,disc_facility,Planet_controversial_flag,Orbital_period[days],delta_orbpererr1,...,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,st_nspec
2,K2-170 b,K2-170,Gaia DR2 2599451994385832448,1,2,0,K2,0,7.576544,0.001841,...,12.668,0.114,-0.114,11.075,0.021,-0.021,12.49590,0.000222,-0.000222,0
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,15.030,0.229,-0.229,12.782,0.022,-0.022,14.85610,0.000306,-0.000306,0
4,Kepler-1398 b,Kepler-1398,Gaia DR2 2128776801985730048,1,2,0,Kepler,0,2.788157,0.000020,...,13.752,0.103,-0.103,12.355,0.021,-0.021,13.66050,0.000228,-0.000228,0
5,Kepler-1547 b,Kepler-1547,Gaia DR2 2104890633423618048,1,1,0,Kepler,0,0.692980,0.000001,...,13.464,0.114,-0.114,11.996,0.020,-0.020,13.40850,0.000189,-0.000189,0
6,TOI-4559 b,TOI-4559,Gaia DR2 6173120554255287808,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,3.965991,0.000314,...,13.115,0.006,-0.006,8.619,0.025,-0.025,12.07970,0.000583,-0.000583,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,KOI-1831 d,Kepler-324,Gaia DR2 2132078120004385152,1,4,0,Kepler,0,34.169000,NaN,...,14.222,0.069,-0.069,12.291,0.025,-0.025,14.09090,0.000217,-0.000217,0
696,HD 137496 b,HD 137496,Gaia DR2 6258810550587404672,1,2,0,K2,0,1.621160,0.000079,...,9.899,0.006,-0.006,8.244,0.036,-0.036,9.77168,0.000601,-0.000601,0
697,Kepler-101 c,Kepler-101,Gaia DR2 2131709921047710336,1,2,0,Kepler,0,6.029760,0.000075,...,13.700,0.069,-0.069,12.011,0.020,-0.020,13.56720,0.000214,-0.000214,0
698,LHS 1678 c,LHS 1678,Gaia DR2 4864160624337973248,2,3,0,Transiting Exoplanet Survey Satellite (TESS),0,3.694284,0.000005,...,12.600,0.200,-0.200,8.264,0.029,-0.029,11.54030,0.000509,-0.000509,0


In [24]:
for planet in kopparu:
    print(planet)

Planet_name
Host_name
gaia_id
Number_of_stars
Number_of_planets
Number_of_moons
disc_facility
Planet_controversial_flag
Orbital_period[days]
delta_orbpererr1
delta_orbpererr2
pl_orbperlim
Orbit_semimajor_axis
delta_orbsmaxerr1
delta_orbsmaxerr2
pl_orbsmaxlim
angular_separation[mas]
delta_angseperr1
delta_angseperr2
pl_angseplim
Planet_radius[Earth]
delta_radeerr1
delta_radeerr2
pl_radelim
Planet_radius[Jupiter]
delta_radjerr1
delta_radjerr2
pl_radjlim
Planet_mass[Earth]
delta_bmasseerr1
delta_bmasseerr2
pl_bmasselim
Planet_mass[Jupiter]
delta_bmassjerr1
delta_bmassjerr2
pl_bmassjlim
Planet_density[g/cm^3]
delta_denserr1
delta_denserr2
pl_denslim
Orbit_eccentricity
delta_orbeccenerr1
delta_orbeccenerr2
pl_orbeccenlim
Planet_insolation_flux[Earth]
delta_insolerr1
delta_insolerr2
pl_insollim
Planet_equilibrium_temperature[K]
delta_eqterr1
delta_eqterr2
pl_eqtlim
Orbital_inclinitation[deg]
delta_orbinclerr1
delta_orbinclerr2
pl_orbincllim
ttv_flag
Ratio_semimajor_stellarradius
delta_ratdor

In [25]:
main_stars = []
for index, planet in kopparu.iterrows():
    spec_type = str(planet['Spectral_type'])
    if 'V' in spec_type and 'I' not in spec_type:
        main_stars.append(planet)

# Convert the list of rows back to a DataFrame
main_stars_df = pd.DataFrame(main_stars)
main_stars_kopparu = kopparu[kopparu.index.isin(main_stars_df.index)]
main_stars_kopparu

,Planet_name,Host_name,gaia_id,Number_of_stars,Number_of_planets,Number_of_moons,disc_facility,Planet_controversial_flag,Orbital_period[days],delta_orbpererr1,...,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,st_nspec
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,15.030,0.229,-0.229,12.782000,0.022,-0.022,14.85610,0.000306,-0.000306,0
7,TOI-836 b,TOI-836,Gaia DR2 6230733559097425152,1,2,0,Transiting Exoplanet Survey Satellite (TESS),0,3.816730,1.000000e-05,...,9.920,0.030,-0.030,6.804000,0.018,-0.018,9.40475,0.000443,-0.000443,0
8,TRAPPIST-1 f,TRAPPIST-1,Gaia DR2 2635476908753563008,1,7,0,Multiple Observatories,0,9.207540,3.200000e-05,...,17.020,0.200,-0.200,10.296000,0.023,-0.023,15.64510,0.001448,-0.001448,0
10,Kepler-1759 b,Kepler-1759,Gaia DR2 2086879189856001024,1,1,0,Kepler,0,3.720230,NaN,...,14.532,0.092,-0.092,12.159000,0.025,-0.025,14.26010,0.000254,-0.000254,0
11,Kepler-1813 b,Kepler-1813,Gaia DR2 2129660087777589248,1,1,0,Kepler,0,2.818160,NaN,...,15.639,0.103,-0.103,13.643000,0.051,-0.051,15.53610,0.001290,-0.001290,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,K2-141 b,K2-141,Gaia DR2 2643952940813536768,1,2,0,K2,0,0.280324,1.500000e-06,...,11.454,0.023,-0.023,8.401000,0.023,-0.023,10.72170,0.000692,-0.000692,0
669,K2-36 b,K2-36,Gaia DR2 3811989156889528320,1,2,0,K2,0,1.422614,3.800000e-05,...,11.726,0.037,-0.037,9.454001,0.025,-0.025,11.52700,0.000904,-0.000904,0
676,TOI-286 b,TOI-286,Gaia DR2 5482316880495405056,1,2,0,Transiting Exoplanet Survey Satellite (TESS),0,4.511724,3.100000e-06,...,9.866,0.005,-0.005,7.851000,0.033,-0.033,9.65064,0.000274,-0.000274,0
688,CoRoT-7 b,CoRoT-7,Gaia DR2 3107267177757848576,1,3,0,CoRoT,0,0.853592,6.000000e-09,...,11.728,0.027,-0.027,9.806000,0.021,-0.021,11.52390,0.000462,-0.000462,0


Para conocer las distancias de los límites de la zona habitable para cada tipo de estrella, se considera la siguiente ecuación paramétrica
$ S_{eff} = S_{eff\odot } + aT_{*} + bT_{*}^2 + cT_{*}^3 + dT_{*}^4$
donde $T_{*} = T_{eff}-5780k.$
Para ello, se recurre a hacer una interpolación lineal de $S_{eff \odot}$ de kopparu  para cada masa de los planetas en el límite Runway_Greenhouse.  

In [ ]:
# Interpolación 
# S_eff_sun = [Recent_venus, Runaway_Greenhouse,Maximum_greenhouse, Early_Mars, 0.1M Runaway_Greenhouse, 5M Runwa]
S_eff_sun = [1.776,1.107, 0.356, 0.320, 1.188, 0.99]
interpolation = np.interp(main_stars_kopparu['Planet_mass[Earth]'], [0.1, 5.0], S_eff_sun[4:])
S_eff_interpolated = np.concatenate([np.array(S_eff_sun[0:3]),interpolation])
# FLujo solar interpolado 
S_eff_interpolated

array([1.776     , 1.107     , 0.356     , 1.08738367, 1.00899184,
       1.15005673, 1.1647249 , 1.08778776, 1.16480571, 1.08819184,
       1.16403796, 1.16403796, 1.16419959, 1.1637551 , 1.08778776,
       1.16367429, 1.16359347, 1.08940408, 1.16290653, 1.11486122,
       1.09102041, 1.09061633, 1.09021224, 1.16133061, 1.09061633,
       1.15935061, 1.02757959, 1.09182857, 1.09182857, 1.15898694,
       1.15890612, 1.15862327, 1.15591592, 1.15551184, 1.15539061,
       1.09465714, 1.06960408, 1.15417837, 1.15417837, 1.15401673,
       1.15042041, 1.15320857, 1.15276408, 1.15276408, 1.09627347,
       1.15122857, 1.15042041, 1.15122857, 1.14880408, 1.15122857,
       1.09667755, 1.14476327, 1.14597551, 1.0978898 , 1.14880408,
       1.1484    , 1.1484    , 1.1484    , 1.05020816, 1.14961224,
       1.14678367, 1.14678367, 1.14678367, 1.14799592, 1.09910204,
       1.09910204, 1.09910204, 1.14112653, 1.14516735, 1.0999102 ,
       1.14355102, 1.10031429, 1.14153061, 1.14112653, 1.14193

In [ ]:
S_eff = np.zeros_like()

In [ ]:


# Example: Fit S_eff vs T_eff for a new planetary mass
T_eff = np.array([2600, 3000, 5780, 7200])  # Stellar temperatures
S_eff_model = np.array([0.35, 0.5, 1.107, 1.45])  # Your model results

# Fit 4th-order polynomial
coefficients = np.polyfit(T_eff - 5780, S_eff_model, 4)
print("a, b, c, d =", coefficients[::-1])  # [d, c, b, a]

a, b, c, d = [ 1.10700000e+00  1.96102250e-04  2.14398179e-08  8.50524842e-12
 -7.50004278e-16]


/tmp/ipykernel_28527/291187221.py:6: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(T_eff - 5780, S_eff_model, 4)
